In [1]:
import numpy as np

import torch
from torch.utils.data import random_split
from torchvision import datasets, transforms
import torch.nn as nn

from tqdm import tqdm

import matplotlib.pyplot as plt

In [2]:
train_transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize((0.3490, 0.3007, 0.2846), (0.2464, 0.2273, 0.2159))
])
val_transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize((0.3664, 0.3186, 0.2949), (0.2424, 0.2254, 0.2134))
])
test_transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize((0.3449, 0.2903, 0.2779), (0.2453, 0.2206, 0.2119))
])

In [3]:
train_dataset = datasets.ImageFolder('./data/train/', transform=train_transform)
val_dataset = datasets.ImageFolder('./data/val/', transform=val_transform)
test_dataset = datasets.ImageFolder('./data/test/', transform=test_transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
nimages = 0
mean = 0.0
var = 0.0
for i_batch, batch_target in enumerate(test_dataloader):
    batch = batch_target[0]
    # Rearrange batch to be the shape of [B, C, W * H]
    batch = batch.view(batch.size(0), batch.size(1), -1)
    # Update total number of images
    nimages += batch.size(0)
    # Compute mean and std here
    mean += batch.mean(2).sum(0) 
    var += batch.var(2).sum(0)

mean /= nimages
var /= nimages
std = torch.sqrt(var)

print(mean)
print(std)

In [117]:
class Classificator(nn.Module):
    def __init__(self):
        super(Classificator, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=11) # after that step: 290x290x32
        self.pool1 = nn.MaxPool2d(kernel_size=7, stride=2) # after that step: 142x142x32
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=6, kernel_size=9) # after that step: 134x134x64
        self.pool2 = nn.MaxPool2d(kernel_size=5, stride=2) # after that step: 65x65x64
        self.conv3 = nn.Conv2d(in_channels=6, out_channels=8, kernel_size=7) # after that step: 59x59x86
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2) # after that step: 29x29x86
        self.conv4 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=5) # after that step: 25x25x86
        self.pool4 = nn.MaxPool2d(kernel_size=3, stride=2) # after that step: 12x12x86
        self.fc1 = nn.Linear(in_features=8 * 12 * 12, out_features=32)
        self.fc6 = nn.Linear(in_features=32, out_features=2)

        self.dropout = nn.Dropout(p=0.3)
        self.leaky_relu = nn.LeakyReLU()
        self.bn1 = nn.BatchNorm2d(4)
        self.bn2 = nn.BatchNorm2d(6)
        self.bn3 = nn.BatchNorm2d(8)
        self.bn4 = nn.BatchNorm2d(8)
        self.bn5 = nn.BatchNorm1d(32)

    def forward(self, x):
        x = self.pool1(self.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool2(self.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool3(self.leaky_relu(self.bn3(self.conv3(x))))
        x = self.dropout(x)
        x = self.pool4(self.leaky_relu(self.bn4(self.conv4(x))))
        x = self.dropout(x)
        x = x.view(-1, 8 * 12 * 12)
        x = self.leaky_relu(self.bn5(self.fc1(x)))
        x = self.dropout(x)
        # x = self.leaky_relu(self.fc3(x))
        # x = self.leaky_relu(self.fc4(x))
        # x = self.leaky_relu(self.fc5(x))
        x = self.fc6(x)

        return x

In [5]:
import copy

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.best_model = None
        self.best_epoch = None

    def __call__(self, epoch, val_loss, model):
        if val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print(f'Early stopping triggered after {epoch} epochs.')
                return True
        else:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_epoch = epoch
        return False

In [120]:
clf = Classificator()
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-4
num_epochs = 150
optimizer = torch.optim.Adam(clf.parameters(), lr=learning_rate)
early_stopping = EarlyStopping(patience=70)

for epoch in tqdm(range(num_epochs)):
    clf.train()
    train_loss = 0
    for data in train_dataloader:
        img, labels = data
        optimizer.zero_grad()
        output = clf(img)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * img.size(0)
    train_loss /= len(train_dataloader.dataset)
    clf.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_dataloader:
            img, labels = data
            output = clf(img)
            loss_eval = criterion(output, labels)
            val_loss += loss_eval.item() * img.size(0)
        val_loss /= len(val_dataloader.dataset)
    print('Epoch [{}/{}], train_loss: {:.4f}, val_loss: {:.4f}'.format(epoch+1, num_epochs, train_loss, val_loss))
    if early_stopping(epoch, val_loss, clf):
        break

from pushbullet import PushBullet
from pywebio.input import *
from pywebio.output import *
from pywebio.session import *
import time

access_token = 'o.WnIMRK2VEcCmjseJDUEy4z6CkzDrciqW'
pb = PushBullet(access_token)
push = pb.push_note('done learning', 'learning finished')

  1%|          | 1/150 [00:16<41:44, 16.81s/it]

Epoch [1/150], train_loss: 0.7298, val_loss: 0.6947


  1%|▏         | 2/150 [00:35<43:32, 17.65s/it]

Epoch [2/150], train_loss: 0.7241, val_loss: 0.6974


  2%|▏         | 3/150 [00:47<37:15, 15.21s/it]

...

Epoch [148/150], train_loss: 0.1444, val_loss: 0.5801


 99%|█████████▉| 149/150 [31:48<00:12, 12.58s/it]

Epoch [149/150], train_loss: 0.1281, val_loss: 0.8737


100%|██████████| 150/150 [32:01<00:00, 12.81s/it]

Epoch [150/150], train_loss: 0.1397, val_loss: 0.6893


In [128]:
correct, total = 0, 0
average_acc = []
for i in range(50):
    clf = Classificator()
    clf.load_state_dict(early_stopping.best_model)
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = clf(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    average_acc.append(100 * correct / total)
print(f'Accuracy on the test set:  {np.mean(average_acc)} %')

Accuracy on the test set:  63.87561803369005 %


In [100]:
clf = Classificator()
clf.load_state_dict(early_stopping.best_model)
torch.save(clf.state_dict(), 'model7697.pth')

In [121]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune

# Prune the model
# prune.random_unstructured(clf.conv1, name="weight", amount=0.3)
# prune.random_unstructured(clf.conv2, name="weight", amount=0.3)
clf = Classificator()
clf.load_state_dict(early_stopping.best_model)

# Calculate importance scores
importance_scores = {}
for name, module in clf.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        importance_scores[name] = module.weight.abs().sum().item()

# Print the importance scores
print(importance_scores)

{'conv1': 38.683963775634766, 'conv2': 54.89545822143555, 'conv3': 67.53501892089844, 'conv4': 56.95692443847656, 'fc1': 542.295654296875, 'fc6': 6.457802772521973}


In [ ]:
clf = Classificator()
clf.load_state_dict(early_stopping.best_model)

prune.random_unstructured(clf.fc2, name="weight", amount=0.2)
# prune.random_unstructured(clf.conv2, name="weight", amount=0.4)
# prune.random_unstructured(clf.fc1, name="weight", amount=0.4)

# Calculate importance scores
importance_scores = {}
for name, module in clf.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        importance_scores[name] = module.weight.abs().sum().item()

# Print the importance scores
print(importance_scores)

In [12]:
from pushbullet import PushBullet
from pywebio.input import *
from pywebio.output import *
from pywebio.session import *
import time

time.sleep(5)
pb = PushBullet(access_token)
push = pb.push_note('done learning', 'learning finished')

In [ ]:
model = torch.load('model6896epochs20.pth')
print(model)